In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2
import pyccl.ccllib as lib
import pyccl as ccl
import pyccl.nl_pt as pt
from astropy.table import Table
import numpy as np

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.22, Omega_b=0.0448, 
    h=0.71, sigma8 = 0.801, n_s= 0.963,w0=-1.00,wa=0.0, Omega_k=0.0)

Om_m = cosmo['Omega_m']
rho_crit = lib.cvar.constants.RHO_CRITICAL
Aia=1.0 #1.5,2
C2=1.0
sigma_epsilon = 0.27

In [ ]:
def Epsilon1_NLA(cosmo,z,A1,rho_crit,sxx,syy):
    gz = ccl.growth_factor(cosmo, 1./(1+z))
    Fact = -1*A1*5e-14*rho_crit*cosmo['Omega_m']/gz
    e1_NLA =  Fact  * (sxx - syy) 
    return e1_NLA

def Epsilon2_NLA(cosmo,z,A1,rho_crit,sxy):
    gz = ccl.growth_factor(cosmo, 1./(1+z))
    Fact = -1*A1*5e-14*rho_crit*cosmo['Omega_m']/gz
    e2_NLA = 2 *Fact* sxy
    return e2_NLA

def Epsilon1_TT(cosmo,z,A2,rho_crit,sxx,syy):
    gz = ccl.growth_factor(cosmo, 1./(1+z))
    Fact =  -5*A2*5e-14*rho_crit*cosmo['Omega_m']/gz**2
    e1_TT =  Fact * (sxx**2 - syy**2) 
    return e1_TT

def Epsilon2_TT(cosmo,z,A2,rho_crit,sxx,syy,sxy):
    gz = ccl.growth_factor(cosmo, 1./(1+z))
    Fact =  -5*A2*5e-14*rho_crit*cosmo['Omega_m']/gz**2
    e2_TT =  2 * Fact * sxy*(sxx+syy)
    return e2_TT


In [ ]:
for i in np.arange(1,6):
    # descriptions=['ra_deg', 'dec_deg', 'shear1', 'shear2', 'weight', 'z_true', 'delta', 's11', 's22', 's12']
    #catalog = Table.read('/global/cscratch1/sd/jharno/IA_Infusion/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(i), format='ascii')  
    #catalog = Table.read('/global/cscratch1/sd/jharno/IA_Infusion/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij_smooth.dat'.format(i), format='ascii')  

    #V0
    #catalog = Table.read('../../GalCat/StageIV_nz/V0/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(i), format='ascii')  
    #catalog = Table.read('../../GalCat/StageIV_nz/V0/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij_smooth.dat'.format(i), format='ascii')  
    catalog = Table.read('../../GalCat/SRD-Y1/random_pos/GalCat_tomo{}_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_smooth0.1Mpc.asc'.format(i), format='ascii')  
    
    #V1
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias.asc'.format(i), format='ascii')  
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc'.format(i), format='ascii')  
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc'.format(i), format='ascii')  
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_1Mpc.asc'.format(i), format='ascii')  

    print('Got Gatalogue')
    
    s11=catalog['col8']
    s22=catalog['col9']
    s12=catalog['col10']
    z=catalog['col6']
    
    e1_NLA=Epsilon1_NLA(cosmo,z,Aia,rho_crit,s11,s22)
    e2_NLA=Epsilon2_NLA(cosmo,z,Aia,rho_crit,s12)
    
    e1_TT=Epsilon1_TT(cosmo,z,C2,rho_crit,s11,s22)
    e2_TT=Epsilon2_TT(cosmo,z,C2,rho_crit,s11,s22,s12)

    random1=np.random.normal(0, sigma_epsilon, e1_NLA.shape)
    random2=np.random.normal(0, sigma_epsilon, e2_NLA.shape)
    
    print('Got Shapes')
    
    epsilon = Table([e1_NLA, e2_NLA, e1_TT, e2_TT, random1, random2], names = ('e1_NLA', 'e2_NLA', 'e1_TT', 'e2_TT', 'e1_noise', 'e2_noise'))

    #epsilon.write('/global/cscratch1/sd/jharno/epsilon_{}.dat'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('/global/cscratch1/sd/jharno/epsilon_{}_smooth.dat'.format(i), format='ascii', overwrite=True)    

    #***
    #V0:
    #epsilon.write('../../GalCat/StageIV_nz/V0/epsilon_{}_0.06GpAM_V2.asc'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('../../GalCat/StageIV_nz/V0/epsilon_{}_0.06GpAM_smooth_V2.asc'.format(i), format='ascii', overwrite=True)    
    epsilon.write('../../GalCat/SRD-Y1/random_pos/epsilon_{}_0.6GpAM_smooth0.1.asc'.format(i), format='ascii', overwrite=True)    

    #***
    #V1
    #epsilon.write('../../GalCat/StageIV_nz/V1/epsilon_{}_0.06GpAM_linear_bias.asc'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('../../GalCat/StageIV_nz/V1/epsilon_{}_0.06GpAM_linear_bias_smooth.asc'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('../../GalCat/StageIV_nz/V1/epsilon_{}_0.6GpAM_linear_bias_smooth.asc'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('../../GalCat/StageIV_nz/V1/epsilon_{}_0.06GpAM_linear_bias_smooth_1Mpc.asc'.format(i), format='ascii', overwrite=True)    

    print('Done tomo',i)

In [ ]:
for i in np.arange(1,6):
    #catalog = np.loadtxt('../../GalCat/StageIV_nz/V1/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc')  
    #catalog = np.loadtxt('../../GalCat/StageIV_nz/V1/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta2.asc')  
    #catalog = np.loadtxt('../../GalCat/SRD-Y1/random_pos/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_smooth0.1Mpc.asc')  
    #catalog = np.loadtxt('../../GalCat/SRD-Y1/random_pos/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_smooth0.5Mpc.asc')  
    #catalog = np.loadtxt('../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_bta1_smooth0.5Mpc.asc')  
    catalog = np.loadtxt('../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_bta2_smooth0.5Mpc.asc')  
    print('Got Gatalogue')

    s11=catalog[:,7]
    s22=catalog[:,8]
    s12=catalog[:,9]
    z=catalog[:,5]

    e1_NLA=Epsilon1_NLA(cosmo,z,Aia,rho_crit,s11,s22)
    e2_NLA=Epsilon2_NLA(cosmo,z,Aia,rho_crit,s12)

    e1_TT=Epsilon1_TT(cosmo,z,C2,rho_crit,s11,s22)
    e2_TT=Epsilon2_TT(cosmo,z,C2,rho_crit,s11,s22,s12)

    random1=np.random.normal(0, sigma_epsilon, e1_NLA.shape)
    random2=np.random.normal(0, sigma_epsilon, e2_NLA.shape)

    print('Got Shapes')

    epsilon = Table([e1_NLA, e2_NLA, e1_TT, e2_TT, random1, random2], names = ('e1_NLA', 'e2_NLA', 'e1_TT', 'e2_TT', 'e1_noise', 'e2_noise'))
    #epsilon.write('../../GalCat/StageIV_nz/V1/epsilon_{}_0.6GpAM_linear_bias_smooth.asc'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('../../GalCat/StageIV_nz/V1/epsilon_{}_0.6GpAM_linear_bias_smooth_bta2.asc'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('../../GalCat/StageIV_nz/V1/epsilon_{}_0.06GpAM_linear_bias_smooth_1Mpc.asc'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('../../GalCat/SRD-Y1/random_pos/epsilon_{}_0.6GpAM_smooth0.1Mpc.asc'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('../../GalCat/SRD-Y1/random_pos/epsilon_{}_0.6GpAM_smooth0.5Mpc.asc'.format(i), format='ascii', overwrite=True)    
    #epsilon.write('../../GalCat/SRD-Y1/lin_bias_pos/epsilon_{}_0.6GpAM_smooth0.5Mpc.asc'.format(i), format='ascii', overwrite=True)    
    epsilon.write('../../GalCat/SRD-Y1/lin_bias_pos/epsilon_{}_0.6GpAM_smooth_bta2_0.5Mpc.asc'.format(i), format='ascii', overwrite=True)    

    print('Done tomo',i)

In [ ]:
np.shape(catalog)

In [ ]:
s11